# Multi-Document Agentic RAG for Quantum Computing 

## Setup Groq API Key

Ensure your Groq API key is stored in the environment variable.

In [1]:
# Initialize the Groq client
from groq import Groq
client = Groq()

## Imports and Utils

In [5]:
# Updated Imports

# Example: Adjust these paths based on the actual structure of the package
from llama_index.readers import SimpleDirectoryReader
from llama_index.indexes import SummaryIndex, VectorStoreIndex, ObjectIndex
from llama_index.agents import FunctionCallingAgentWorker, AgentRunner
from llama_index.parsers import TokenTextSplitter
from llama_index.tools import QueryEngineTool, FunctionTool
from llama_index.llms import LLM, ChatMessage, MessageRole
from llama_index.embeddings import HuggingFaceEmbedding

from pathlib import Path
from typing import List, Dict

import nest_asyncio
nest_asyncio.apply()

ImportError: cannot import name 'SimpleDirectoryReader' from 'llama_index' (unknown location)

In [ ]:
# Utility Functions

def get_vector_tool(nodes: List, algo: str, embed_model: any):
    '''Creates a vector index tool that performs vector search.'''
    
    vector_index = VectorStoreIndex(nodes, embed_model=embed_model)
    
    def vector_query(query: str) -> str:
        """Perform a vector search over an index.
    
        query (str): the string query to be embedded.
        """
        query_engine = vector_index.as_query_engine(similarity_top_k=4)
        response = query_engine.query(query)
        return str(response)
    
    vector_query_tool = FunctionTool.from_defaults(
        name=f"{algo}_vector_tool",
        fn=vector_query
    )

    return vector_query_tool

def get_summary_tool(nodes: List, algo: str, llm: any, embed_model: any):
    '''Creates a summary index tool that performs summarization.'''
    
    summary_index = SummaryIndex(nodes, embed_model=embed_model)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
        llm=llm
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"{algo}_summary_tool",
        query_engine=summary_query_engine,
        description=(
            f"Useful if you want to get a summary or explanation of {algo}"
        ),
    )

    return summary_tool

def get_tools(documents: List, llm: any, embed_model: any) -> List:
    '''Returns vector index and summary index tools for the provided documents.'''

    tools = []
    for document in documents:
        # Get the document name
        doc_name = Path(document.metadata.get('filename', 'document')).stem

        # Split text into chunks
        splitter = TokenTextSplitter(
            chunk_size=64,
            chunk_overlap=10,
            separator=" ",
        )
        nodes = splitter.get_nodes_from_documents([document])
        if len(nodes) <= 1:
            raise ValueError(f'Number of generated nodes is less than or equal to 1. Check the document {document}')

        # Get summary and vector index tool
        vector_tool = get_vector_tool(nodes=nodes, algo=doc_name, embed_model=embed_model)
        summary_tool = get_summary_tool(nodes=nodes, algo=doc_name, llm=llm, embed_model=embed_model)

        # Store tools
        tools.extend([vector_tool, summary_tool])

    return tools


## Implementation

In [ ]:
# Load documents from the PDF folder

documents = SimpleDirectoryReader('path_to_your_pdf_folder').load_data()

# Define the GroqLLM class
from llama_index.llms.base import LLM, ChatMessage, MessageRole
from typing import List

class GroqLLM(LLM):
    def __init__(self, client, model_name, temperature=1.0, max_tokens=1024, top_p=1.0):
        self.client = client
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p

    def chat(self, messages: List[ChatMessage]) -> str:
        # Convert ChatMessages to the format expected by Groq client
        groq_messages = []
        for message in messages:
            if message.role == MessageRole.USER:
                groq_messages.append({'role': 'user', 'content': message.content})
            elif message.role == MessageRole.SYSTEM:
                groq_messages.append({'role': 'system', 'content': message.content})
            elif message.role == MessageRole.ASSISTANT:
                groq_messages.append({'role': 'assistant', 'content': message.content})
        
        # Call the Groq client
        completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=groq_messages,
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            top_p=self.top_p,
            stream=False,
            stop=None,
        )
        # Return the assistant's reply
        return completion.choices[0].message.content

# Initialize the LLM
model_name = "llama3-8b-8192"
llm = GroqLLM(client, model_name)

# Initialize the embedding model
embed_model = HuggingFaceEmbedding()

# Get the tools
tools = get_tools(documents=documents, llm=llm, embed_model=embed_model)

In [ ]:
# Create object index and retriever for tools
obj_index = ObjectIndex.from_objects(
    tools,
    index_cls=VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever()

In [ ]:
# Initialize agents
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
# Query agent
response = agent.query(
    "Tell me about Quantum Circuits"
)
print(response)

## References

[Building Agentic RAG with LlamaIndex](https://www.deeplearning.ai/short-courses/building-agentic-rag-with-llamaindex/)